# Practical Session 3: Running ROS in a Docker container

## Laboratorio de Robótica 
### Grado en Ingeniería Electrónica, Mecatrónica y Robótica
### Universidad de Sevilla

## Objectives

In this practical session, we will learn the basics of Docker containers and how to use them to run ROS (Robot Operating System). ROS is a set of software libraries and tools that help you build robot applications, including hardware drivers, state-of-the-art algorithms, robotics simulators, powerful developer tools, and much more. ROS is open-source and has a strong support from the robotics community, so you can find a wide spectrum of available packages for your robotics application. Although ROS can run on different platforms, its most widely used and supported platform is Ubuntu. One option would be to install Ubuntu and ROS on your computer, but in this course we will use a more powerful tool, which is Docker containers. In particular, in this session, we'll learn a bit more about:

+ Using Docker for application development.
+ Running ROS on a Docker container.
+ Simulating our mobile robot Turtlebot 3 in the Gazebo simulator.

## Docker

[Docker](https://www.docker.com/) is an open platform for developing, shipping, and running applications. It enables you to separate your applications from your infrastructure so you can deliver software quickly. Docker allows developers to easily deploy their applications in a _**container**_ running on a host operating system. The container will package and run your application in an isolated environment that can be based on a different operating system. The key benefit of Docker is that it allows users to package an application with all of its dependencies into a standardized unit for software development. Unlike virtual machines, containers do not have high overhead and hence enable more efficient usage of the underlying system and resources. Thus, you can run many containers simultaneously on a given host. As they are lightweight and contain everything needed to run the application, you don't need to rely on what's installed on the host. Moreover, you can share containers while you work, and be sure that everyone you share with gets the same container that works in the same way.


### Main concepts


#### Images

An image is a read-only template with instructions for creating a Docker container. Often, an image is based on another image, with some additional customization. For example, you may build an image which is based on an Ubuntu image, and install ROS and your robotics application on top, as well as the configuration details needed to make your application run. An image is a standardized package that includes all the configuration files, binaries and libraries to run a container. Images are immutable, once created, they cannot be modified. You can only make a new image or add changes on top of it. Moreover, images are composed of layers, and each layer represents a set of file system changes that add, remove, or modify files.


You might use images created by others and published in a registry or create your own images. To build your own image, you create a __Dockerfile__ with a simple syntax for defining the steps needed to create the image and run it. Each instruction in a Dockerfile creates a layer in the image. When you change the Dockerfile and rebuild the image, only those layers which have changed are rebuilt. This is part of what makes images so lightweight, small, and fast, when compared to other virtualization technologies.


#### Docker registries
A Docker registry stores Docker images. [Docker Hub](https://hub.docker.com) is a public registry with over 100,000 images that you can run locally created by developers. By default, Docker searches for images on Docker Hub, but you could also run your own private registry.


#### Containers
A container is a runnable instance of an image. You can create, start, stop, or delete a container using the Docker GUI or CLI. You can run multiple containers from the same image, connect them to the same (or different) network, attach storage to them, etc. By default, a container is relatively well isolated from other containers and its host machine, but this level of isolation can also be configured. 

For further information about how Docker works, check its [documentation](https://docs.docker.com/get-started).


### Installation

__Windows__

Windows Subsystem for Linux (WSL) allows you to run a full Linux environment directly on Windows without a virtual machine. It provides native access to Linux tools and Docker, so we will use to work with Docker in Windows. If not installed, you can install WSL on your Windows machine by opening a PowerShell console as Administrator and running:


In [ ]:
wsl --install

This command will install WSL and the latest Ubuntu distribution (by default). 

__Note 1:__ If you find issues when starting WSL, you may need to enable virtualization features in your computer BIOS, VT-x y VT-d for Intel, AMD-V y AMD-Vi for AMD. Make also sure that Hyper-V is activated. Hyper-V is a Microsoft virtualization tool that comes installed with Windows and is used by WSL. VirtualBox or VMWare have their on hypervisor and may switch off Hyper-V for compatibility. If you find issues running WSL, make sure that Hyper-V is activated. Run `bcdedit | findstr "hypervisorlaunchtype"` to check whether this feature is off. You can activate it with `bcdedit /set hypervisorlaunchtype auto`.

__Note 2:__ You could also install Docker Desktop for Windows with the [installer](https://docs.docker.com/get-started/get-docker), which uses a WSL backend to run Docker. You'll always have to start the Docker Desktop application in the background before using Docker commands. This option is not recommended because it has limitations when configuring Docker containers that need to access a local network.

The following WSL commands can be helpful in a PowerShell console:

| Command | Description |
|----------|--------------|
| `wsl -l -o` | Show available distros online |
| `wsl -l -v` | List installed distros with version info |
| `wsl --install -d Ubuntu-24.04` | Install a specific distro, e.g., Ubuntu 24.04 |
| `wsl --set-default Ubuntu-24.04` | Set a distro as the default |
| `wsl` | Launch the default distro |
| `wsl -d Ubuntu-24.04` | Launch a specific distro |
| `wsl --terminate Ubuntu-24.04` | Terminate a running distro |
| `wsl --shutdown` | Stop all WSL instances |

If it is not already installed, install the Ubuntu distro with: `wsl --install -d Ubuntu`. While installing, you will be asked to provide a username and password. Once you start your WSL Ubuntu, install the following package that will be used later: `sudo apt update && sudo apt install x11-server-utils`. Then install Docker Engine for Ubuntu. Follow the instructions [here](https://docs.docker.com/engine/install/ubuntu/), in Section "Install using the apt repository". Then follow [post-install instructions for Linux](https://docs.docker.com/engine/install/linux-postinstall).

__Note:__ Once your Ubuntu distro is configured in WSL, you can run all your docker-related commands from there. 

By default, WSL uses an internal virtual network. Mirrored mode allows your WSL instance to share the same network interface and IP range as Windows, enabling direct communication with other devices on the same LAN (e.g., a robot to control). If your WSL is version 2.7.0 or higher, activate the WSL mirrored mode by typing in your PowerShell console: `wsl --setnetworkingmode mirrored`. If this command does not exist, open (or create) the file `C:\Users\<YOUR_USERNAME>\.wslconfig` and write the following inside:

In [ ]:
[wsl2]
networkingMode=mirrored
dnsTunneling=true

Restart WSL: `wsl --shutdown`. Reopen your Ubuntu distro and check your network:

In [ ]:
ip addr show

You should now see an IP address in the same range as your Windows host.

__macOS__

Install Docker Desktop for macOS with the [installer](https://docs.docker.com/get-started/get-docker). You may need to start the Docker Desktop application in the background before using Docker commands in the terminal.

__Ubuntu__

Install Docker Engine for Ubuntu (Ubuntu 20.04 LTS or newer is required). Follow the instructions [here](https://docs.docker.com/engine/install/ubuntu/), in Section "Install using the apt repository". Then follow [post-install instructions for Linux](https://docs.docker.com/engine/install/linux-postinstall).

### Using Docker containers

In this section, we are going to learn the basic commands to run Docker containers. You can find a cheat sheet with these main commands [here](https://docs.docker.com/get-started/docker_cheatsheet.pdf). First, let's download a simple test Docker image:

In [ ]:
docker pull hello-world 


The `pull` command allows you to pull any image which is present in the official registry of docker, Docker hub. By default, it pulls the latest image, but you can also specify the image version. You can use the command `docker images` to see a list of all images downloaded on your system. Let's now run a Docker container based on this image: 

In [ ]:
docker run hello-world


This creates a new container from the image `hello-world` and starts that container. If it were not present on the system, the image is first pulled. The `docker ps` command shows you all the containers that are currently running; you can use `docker ps -a` to show also not running containers. `hello-world` is a quite simple image that only runs a binary file that prints a message. You should have seen that message after running the container. 

Instead of using the container ID (which looks like an arbitrary number) to refer to a certain container, you can also specify a container name before the image name `docker run --name <container_name> <image_name>`. Let's run a more interesting container:

In [ ]:
docker run --name ubuntu_container -it ubuntu bash

If it is not on your system, the previous command will first pull the `ubuntu` image, which replicates an Ubuntu file system. Using the `run` command with the `-it` flag moves you into an interactive terminal session where you can execute commands inside the container. The `bash` command is specified as the first command for that container, so it will be as if you were in a new bash terminal separate from your host machine and inside the container. If you don't specify `bash` (or another initial command) when running the container, Docker will run the default command defined in the container's image. 

Try to run some Linux commands inside the container to check the Ubuntu file system, e.g., `ls` and `cd`. Then open another terminal in your host machine and list your containers with `docker ps -a`. Note that now your running container is named `ubuntu_container`, which is the name you specified. 

So far, you have a running container with an interactive terminal session to execute commands inside. However, you may need more than one terminal to run multiple commands in parallel in the same container. For that, open another terminal in your host machine and type:

In [ ]:
docker exec -it ubuntu_container bash

The previous command opens a new bash shell inside your running container `ubuntu_container`. At some point, you may want to stop a container because it crashed or to switch to another one. You may also want to start the container again later. To start or stop an existing container, use `docker start|stop <container_name> (or <container_ID>)`.

Docker containers occupy space on your host's disk, so it is good practice to clean up containers once you are done with them. List your containers with `docker ps -a` and use the `docker rm <container_name` command to remove them. You can also delete images that you no longer need by running `docker rmi <image_name>`. For instance, if you had the following containers, once stopped, you could remove them with the command `docker rm ubuntu_container 8987409a69d4`: 

<figure style="text-align:center">
  <img src="images/docker_terminal.png" alt="" width=1100>
  <figcaption>Fig. 1: Example Docker container list.</figcaption>
</figure>

## Running ROS in a Docker container

In this section, we will set up and run our Docker container with ROS. You can build a customized Docker image from a Dockerfile with `docker build -t <image_name>`. A Dockerfile is a simple text file that contains a list of commands that Docker calls while creating an image. It's a simple way to automate the image creation process. The commands you write in a Dockerfile are almost identical to their equivalent Linux commands.

<figure style="text-align:center">
  <img src="images/dockerfile.png" alt="" width=600>
  <figcaption>Fig. 2: Docker working scheme.</figcaption>
</figure>

Have a look at the Dockerfile in the _docker_ folder of the course repository:

In [ ]:
FROM osrf/ros:noetic-desktop-full

SHELL ["/bin/bash", "-c"] 

# Required ROS packages  
RUN sudo apt update && apt install -y ros-noetic-joy ros-noetic-teleop-twist-joy \
    ros-noetic-teleop-twist-keyboard ros-noetic-laser-proc \
    ros-noetic-rgbd-launch ros-noetic-rosserial-arduino \
    ros-noetic-rosserial-python ros-noetic-rosserial-client \
    ros-noetic-rosserial-msgs ros-noetic-amcl ros-noetic-map-server \
    ros-noetic-move-base ros-noetic-urdf ros-noetic-xacro \
    ros-noetic-compressed-image-transport ros-noetic-rqt* ros-noetic-rviz \
    ros-noetic-gmapping ros-noetic-navigation ros-noetic-interactive-markers \
    ros-noetic-vision-msgs

# ROS packages for Turtlebot3 robot     
RUN sudo apt install -y ros-noetic-dynamixel-sdk \
    ros-noetic-turtlebot3-*

# We also install Git and file editors, just in case
RUN sudo apt install -y git vim nano iputils-ping net-tools

# MESA drivers for hardware acceleration graphics (Gazebo and RViz)
RUN sudo apt -y install libgl1-mesa-glx libgl1-mesa-dri && \
    rm -rf /var/lib/apt/lists/*

# Create a user provided as build argument
ARG USER_NAME=student
ARG USER_ID=1000
ARG GROUP_ID=${USER_ID}

RUN groupadd -g ${GROUP_ID} ${USER_NAME} && \
    useradd -m -u ${USER_ID} -g ${GROUP_ID} -s /bin/bash ${USER_NAME} && \
    echo "${USER_NAME} ALL=(ALL) NOPASSWD:ALL" >> /etc/sudoers

# Source the ROS installation
RUN echo "source /opt/ros/noetic/setup.bash" >> /home/${USER_NAME}/.bashrc && \
    echo "export TURTLEBOT3_MODEL=burger" >> /home/${USER_NAME}/.bashrc

# Working directory inside our container
USER ${USER_NAME}
WORKDIR /home/${USER_NAME}/lab_rob_shared

We will use that Dockerfile to build our image with ROS. Move to the folder containing the Dockerfile and type:

In [ ]:
docker build --build-arg USER_UID=$(id -u) --build-arg USER_GID=$(id -g) --build-arg USER_NAME=$(id -un) -t lab_rob_image .

With the previous command, the container will inherit your user name and id from the host machine (or from the Ubuntu distro, if you are using WSL).
  
__Note:__ In case no user name is specified when building the image, the Dockerfile will create a default user named `student`.

The above command will build a new image called `lab_rob_image`. The dot indicates that the Dockerfile is in the current repository. The instructions in the Dockerfile are used to create the image. The following type of commands are included:

+ `FROM`: It specifies a base image to initialize the first layer. We use an official image with a Linux file system and ROS Noetic installed.
+ `SHELL`: It specifies the default shell to run subsequent commands. 
+ `RUN`: It runs a script or command. In particular, our Dockerfile uses it to install a series of additional ROS packages and drivers that will be necessary.
+ `ARG`: It specifies a build-time argument. A default value can be set in case the argument is not provided in build time.
+ `USER`: It specifies the user name who will run the containers of this image by default. 
+ `WORKDIR`: It sets the working directory for any following instructions in the Dockerfile. The container will start in that directory. If the directory specified by WORKDIR doesn't exist, it will be created. 

You can learn all possible instructions in Dockerfiles [here](https://docs.docker.com/reference/dockerfile/). Other common instructions not used in our Dockerfile are the following:

+ `COPY`: It copies new files or directories and adds them to the file system of the image.
+ `CMD`: It tells the container which command to run when started. This command can be overriden when launching a container.
+ `ENV`: It sets an environment variable. This variable will be accessible during the build process and once the container is run.

### Running the container with a bind mount and GUI support

Running our container and executing a terminal sounds cool, but there are a couple of additional features that would be quite helpful: 1) how to exchange data with the container; and 2) how to run graphical applications in the container. This would allow us to load our code into the container and then retreive any data generated within it. Additionally, we could run applications that use graphics, such as simulators or visualization tools. Fortunately, Docker provides solutions for both issues. 

Docker allows you to create a virtual volume for a container, where your data can persist after stopping the container. A volume mount is a great choice when you need somewhere persistent to store your application data. A __bind mount__ is another type of mount, which allows you to share a directory from the host's file system with the container. For instance, you can use this shared directory to store your workspace with your code. The container will see the changes you make to the code immediately, as soon as you save a file, but you won't need to copy that code into the container. You can check more information about bind mounts in Docker [here](https://docs.docker.com/get-started/workshop/06_bind_mounts/). 

In short, you can create a link between a directory in the host machine and a directory in the container by adding the flag `--mount` to the Docker running command:

In [ ]:
docker run -it --mount type=bind,source=/home/[USER]/lab_rob_shared,target=/home/[USER]/lab_rob_shared lab_rob_image bash

The previous command creates a link between the directory `lab_rob_shared` in the host machine and another one in the container, mounted on `/home/[USER]/lab_rob_shared`. Therefore, that shared directory will be accessible from both the container and the host.   

__Note 1:__ The previous command assumes the existence of the host directory `/home/[USER]/lab_rob_shared`. Make sure to create first that shared directory in your computer, replacing `[USER]` with your own user's name. If you are using WSL in Windows, the path could be something like `/mnt/c/Users/[USER]/lab_rob_shared`.

__Note 2:__ The idea is to use the container to compile and execute your ROS code, but to edit it on your host computer. 


#### GUI support

Regarding graphical applications, we can enable the use of GUIs within Docker containers by means of the X server. The X server is a windowing system for bitmap displays commonly in Linux operating systems. We can connect a container to the host's X server for display. 

__Note:__ There are other options to run Docker containers with GUI support. In case you need more information about that, check it [here](https://wiki.ros.org/es/docker/Tutorials/GUI). 

__Ubuntu and Windows__

If your host machine is running Ubuntu (natively or through WSL), there is already an X server, so we just need to add some flags when running the Docker container:

In [ ]:
xhost +local:docker
docker run -it \
    --rm \
    --env="DISPLAY=$DISPLAY" \
    --env="QT_X11_NO_MITSHM=1" \
    --volume="/tmp/.X11-unix:/tmp/.X11-unix" \
    --name lab_rob_container \
    --net=host \
    --privileged \
    --mount type=bind,source=/home/[USER]/lab_rob_shared,target=/home/[USER]/lab_rob_shared \
    lab_rob_image \
    bash

With the previous command, we would run the container connected to the host's X server, to run GUI applications, and with a bind mount to share a directory with the host machine. First, the `xhost` command adjusts the permissions of the X server on your host.
Then we call `docker run` to run a container called `lab_rob_container` from our previously created image `lab_rob_image`. The first three flags are required to enable graphical applications. The `--mount` flag at the end is to create the shared folder ` lab_rob_shared` between the container and the host.  

Instead of writing or copying the previous command in the terminal each time you want to run the container, we provide a
bash script called `run_container_linux.bash` in the `docker` directory of the course repository. Before running the script, you should edit it to write the correct path of your shared directory in the mount flag and change the script execution permissions:

In [ ]:
chmod a+x run_container_linux.bash
./run_container_linux.bash

__macOS__

On macOS, you need to install an X server. XQuartz is a common choice; you can install it with: `brew install --cask xquartz`. Once you have an X server installed, start it before running your container. Then you can run the Docker container as follows:

In [ ]:
xhost +localhost
docker run -it \
    --rm \
    --env="DISPLAY=host.docker.internal:0" \
    --name lab_rob_container \
    --net=host \
    --privileged \
    --mount type=bind,source=/home/[USER]/lab_rob_shared,target=/home/[USER]/lab_rob_shared \
    lab_rob_image \
    bash

With the previous command, we would run the container connected to the host's X server, to run GUI applications, and with a bind mount to share a directory with the host machine. We call `docker run` to run a container called `lab_rob_container` from our previously created image `lab_rob_image`. The first flag is required to enable graphical applications. The `--mount` flag at the end is to create the shared folder `lab_rob_shared` between the container and the host.  

Instead of writing or copying the previous command in the terminal each time you want to run the container, we provide a script in the `docker` directory of the course repository, `run_container_mac.bash`. Before running the script, you should edit it to write the correct path of your shared directory in the mount flag and change the script execution permissions:

In [ ]:
# For macOS
chmod a+x run_container_mac.bash
./run_container_mac.bash

__Exercise 1__

Create in your machine your shared directory called `lab_rob_shared`. Then create the ROS Docker image from the provided Dockerfile and use the aforementioned scripts to run a Docker container with a bind mount and GUI support. In the container, run a ROS simulation of the Turtlebot3 robot with the command: `roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch`. Finally, open another terminal within the same container and run a ROS teleoperation interface to move the robot with the keyboard: `roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch`. 

__Note:__ The provided script is only used one time, to start the container. Once you have your container running, if you want to open additional terminals in this container, use the command: `docker exec -it lab_rob_container bash`. 

## Summary

In this practical session, you should have learned the following:
+ How to install Docker and how to use it to create images and run containers.
+ How to run our ROS container with GUI support and a shared directory with your code repository.